LIBRARY

In [2]:
import requests
import pandas as pd
import requests
import time
import urllib
import glob
from bs4 import BeautifulSoup
import json
import csv
from pandasql import sqldf
from sqlalchemy import create_engine
from functools import *
import mysql.connector as msql
from mysql.connector import Error
import numpy as np
import pymysql
import time
import threading
import queue
from sqlalchemy import text

**INTERNAL FUNCTION**

In [3]:
 ### Find value of key in nested json 
def find_value_of_key(value, key):
    val = None
    if isinstance(value,dict):
        if key in value:
            return value[key]
        for i in value.items():
            val = find_value_of_key(i[1], key)
            if val is not None:
                return val
    if isinstance(value,list):
        for i in value:
            val = find_value_of_key(i, key)
    return val

**CRAWLING CATEGORIES FROM ONE PAGE**

In [40]:
def get_product_catagories(url):
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    response = requests.get(url=url, headers=http_headers)
    parser = BeautifulSoup(response.text, 'html.parser', )
    product_box = parser.findAll('script')
    product_catagories = []
    for product in product_box:
        try:
            item = json.loads(str(product.contents[0]))
        except:
            item = {}
        product_val = find_value_of_key(item,"breadcrumbs")
        if product_val is not None:
            [product_catagories.append(i) for i in product_val if i["category_id"] != 0] 
    return product_catagories


### Testing the Functional
get_product_catagories('https://tiki.vn/combo-2-tui-nuoc-xa-vai-comfort-giu-mau-ben-vai-mot-lan-xa-huong-ban-mai-tui-3-8l-p134516825.html?itm_campaign=tiki-reco_UNK_DT_UNK_UNK_tiki-listing_UNK_p-category-mpid-listing-v1_202306170600_MD_batched_PID.134516826&itm_medium=CPC&itm_source=tiki-reco&spid=134516826')


[{'url': '/do-choi-me-be/c2549',
  'name': 'Đồ Chơi - Mẹ & Bé',
  'category_id': 2549},
 {'url': '/cham-soc-nha-cua/c15078',
  'name': 'Chăm sóc nhà cửa',
  'category_id': 15078},
 {'url': '/giat-giu-cham-soc-quan-ao/c4387',
  'name': 'Giặt giũ & Chăm sóc quần áo',
  'category_id': 4387},
 {'url': '/nuoc-xa-vai/c4403', 'name': 'Nước xả vải', 'category_id': 4403}]

**GET ID OF PRODUCT**

In [5]:
def get_max_page_number(name:str, categories_id:int):
    http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
    }
    url = f'https://tiki.vn/{name}/c{categories_id}'
    def find_max_tag(a,b):
        if isinstance(a,int):
            a_num = a
        else:
            a_tag = a.find('a')
            a_num = a_tag.get('data-view-label') if a_tag.get('data-view-label') is not None else 0
        b = b.find('a')
        b_num = b.get('data-view-label') if b.get('data-view-label') is not None else 0
        return max(int(a_num), int(b_num))
    
    div_tag = None
    while(div_tag is None):
        response = requests.get(url=url, headers=http_headers)
        parser = BeautifulSoup(response.text, 'html.parser', )
        div_tag = parser.find("div", {"data-view-id":"product_list_pagination_container"})   
    # find all 
    ul_tag = div_tag.find('ul')
    li_tag = ul_tag.findAll('li')
    return reduce(find_max_tag,li_tag)

### Testing the Functional
get_max_page_number("Điện thoại Smartphone", 1795)

18

**CRAWLING ALL ID FROM CATEGORIES**

In [6]:
def crawling_product_id(name:str, categories_id:int, page_number:int):
    http_headers = {
        'User-Agent':  'PostmanRUntime/7.29.2',
    }
    product_list = set()
    for i in range(page_number):
        url =f'https://tiki.vn/{name}/c{categories_id}?page{i}'
        response = None
        while(response is None):
            response = requests.get(url=url, headers=http_headers)
        parser = BeautifulSoup(response.text, 'html.parser')
    
        # find all 
        product_box = parser.findAll('a')
        product_box = [[]] + list(product_box)
        res = set(reduce(get_product_id,product_box))
        product_list.update(res)
    return list(product_list)

 
def get_product_id(pre_val,product):
    data = product.get("data-view-content")
    if data is not None:
        item = json.loads(data)
        id = item['click_data'].get('id')
        if id is not None:
            pre_val.append(id)
    return pre_val

### Testing the Functional
crawling_product_id("Điện thoại Smartphone", 1795,12)

[203763209,
 252427658,
 250109584,
 220680981,
 221274391,
 177343384,
 197216279,
 197216291,
 194034217,
 247451178,
 230155305,
 147969070,
 250113073,
 247823537,
 120295859,
 184696247,
 187266104,
 212040697,
 263041214,
 197966909,
 197211969,
 216174917,
 198648645,
 203278278,
 57809866,
 184059211,
 197214029,
 253269077,
 247925719,
 200995800,
 252423003,
 203864283,
 184036446,
 189658085,
 198671718,
 171897574,
 247730281,
 170497385,
 258373871,
 244308977,
 191477618,
 249890677,
 142545270,
 173764601,
 132068858,
 216173180,
 91612670]

**CALL API TO GET PRODUCT INFORMATION, CATEGORIES**

In [7]:
def get_data(product_id:int):
    http_headers = {
    'User-Agent':  'PostmanRUntime/7.29.2',
    }
    url = f'https://tiki.vn/api/v2/products/{product_id}'
    response = requests.get(url, headers = http_headers)
    result = response.json()
    categories = result.get('breadcrumbs')
    return response.json(), categories
    
data, categories = get_data(177746939)


In [8]:
categories

[{'url': '/dien-thoai-may-tinh-bang/c1789',
  'name': 'Điện Thoại - Máy Tính Bảng',
  'category_id': 1789},
 {'url': '/dien-thoai-smartphone/c1795',
  'name': 'Điện thoại Smartphone',
  'category_id': 1795},
 {'url': 'https://tiki.vn/dien-thoai-samsung-galaxy-a73-5g-8gb-128gb-hang-chinh-hang-da-kich-hoat-bao-hanh-dien-tu-p177746939.html',
  'name': 'Điện Thoại Samsung Galaxy A73 5G (8GB/128GB) - Hàng chính hãng - ĐÃ KÍCH HOẠT BẢO HÀNH ĐIỆN TỬ',
  'category_id': 0}]

**DUMP DATA TO FILE AND EXTRACT CATEGORIES**

In [9]:
### DATA FEILD 
f_brand = ['id','name','slug'] ##### brand
f_brand_crawl = [{'brand':f_brand}]
f_seller = ['id','name','store_id','is_best_store'] ##### current_seller, other_seller
f_seller_crawl = [{'current_seller': f_seller},{'other_seller': f_seller}]
f_product = ['id','name', 'short_url', 'short_description', 'price', 'list_price', 'original_price', 'discount', 'discount_rate',
            'rating_average', 'inventory_status', 'inventory_type', 'is_fresh', 'seller','is_gift_card','data_version',
            'brand', 'current_seller','other_sellers','inventory','breadcrumbs','amplitude']
f_categories = ['url', 'name', 'category_id']

In [10]:
#### DUMP HEADER TO FILE
def convert_product_feild(feild):
    res = []
    for x in feild:
        if isinstance(x,dict):
            for k,v in x.items():
                mini_res = [k +'_'+ i for i in v]
            res = res  + mini_res
        else:
            res.append(x)
    return res

product_header = convert_product_feild(f_product)
with open('Data/product.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(product_header)

with open('Data/categories.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(f_categories)

with open('Data/sellers.csv', 'w') as f:
    writing_data= csv.writer(f, delimiter=',')
    writing_data.writerow(f_seller)

In [11]:
#### GET DATA AND DUMP TO FEILD
def extract_data_to_feild(data, feild, res = {}):
    for i in feild:
            res[i] = extract_data(data,i)
    return res
    
def extract_data(data,key):
    value = data.get(key)
    if value is None:
        value = find_value_of_key(data,key)
    return value
####TESTING FUNCTIONS
data, categories = get_data(192455760)
extract_data_to_feild(data, f_product)

{'id': 192455760,
 'name': 'Máy tính Khoa Học Casio FX-570VN PLUS - 2 (TL)',
 'short_url': 'https://tiki.vn/product-p192455760.html?spid=192455761',
 'short_description': '[BẢO HÀNH CHÍNH HÃNG 7 NĂM] Máy Tính Học Sinh Casio FX-570VN PLUSThông số kỹ thuậtKích thước: Màn hình hiển thị: Trọng lượng sản phẩm: Bảo hành: Thương hiệu: Sản xuất tại: ,\xa0,\xa0Hình thức s...',
 'price': 525000,
 'list_price': 525000,
 'original_price': 525000,
 'discount': 0,
 'discount_rate': 0,
 'rating_average': 5,
 'inventory_status': 'available',
 'inventory_type': 'backorder',
 'is_fresh': False,
 'seller': None,
 'is_gift_card': False,
 'data_version': 3300,
 'brand': {'id': 26150, 'name': 'Casio', 'slug': 'casio'},
 'current_seller': {'id': 204329,
  'sku': '2769719700908',
  'name': 'Nhà Sách Tú Anh',
  'link': 'https://tiki.vn/cua-hang/nha-sach-tu-anh',
  'logo': 'c1/bd/2c/768bc3bc6186cef1ae341d6f32d0a79c.jpg',
  'price': 525000,
  'product_id': '192455761',
  'store_id': 191362,
  'is_best_store': Fa

In [12]:
#### pandas extract data
def extract_product_data(data):
    c_seller_id = 0
    if data['current_seller'] is not None:
        c_seller_id = data['current_seller']['id']
    o_seller_id = None
    if data['other_sellers'] is not None:
        o_seller_id = [item['id'] for item in data['other_sellers']]
    brand_id = data['brand']['id']
    inventory_st  = data['inventory']
    amplitude = data['amplitude']

    del data['amplitude']
    del data['inventory']

    data['other_sellers'] = o_seller_id
    data['current_seller'] = c_seller_id
    data['brand'] = brand_id
    data["is_fresh"] = 1 if data["is_fresh"] == True else 0
    data["is_gift_card"] = 1 if data["is_gift_card"] == True else 0

    data['categories'] = data['breadcrumbs']
    del data['breadcrumbs']
    if amplitude is not None:
        data.update(amplitude)
    if inventory_st is not None:
        data.update(inventory_st)
    
    df = pd.json_normalize(data)
    return df

data, categories = get_data(123345348)
f_data = extract_data_to_feild(data, f_product)
df = extract_product_data(f_data)
pd.DataFrame(df)
f_data

{'id': 123345348,
 'name': 'Apple iPhone 12',
 'short_url': 'https://tiki.vn/product-p123345348.html?spid=70766427',
 'short_description': 'Điện Thoại iPhone 12iPhone 12 đẩy nhanh mọi tác vụ với mạng 5G siêu nhanh.1 A14 Bionic, chip nhanh nhất trên điện thoại thông minh. Hệ thống camera kép mới. Với màn hình Super Retina XDR tuyệt đẹp, n...',
 'price': 15990000,
 'list_price': 18990000,
 'original_price': 18990000,
 'discount': 3000000,
 'discount_rate': 16,
 'rating_average': 5,
 'inventory_status': 'available',
 'inventory_type': 'instock',
 'is_fresh': 0,
 'seller': None,
 'is_gift_card': 0,
 'data_version': 3300,
 'brand': 17827,
 'current_seller': 1,
 'other_sellers': [],
 'categories': [{'url': '/dien-thoai-may-tinh-bang/c1789',
   'name': 'Điện Thoại - Máy Tính Bảng',
   'category_id': 1789},
  {'url': '/dien-thoai-smartphone/c1795',
   'name': 'Điện thoại Smartphone',
   'category_id': 1795},
  {'url': 'https://tiki.vn/apple-iphone-12-hang-chinh-hang-p123345348.html',
   'name'

In [13]:
data, categories = get_data(192455760)
f_data = extract_data_to_feild(data, f_product)
df = extract_product_data(f_data)
list(df.columns.values)

['id',
 'name',
 'short_url',
 'short_description',
 'price',
 'list_price',
 'original_price',
 'discount',
 'discount_rate',
 'rating_average',
 'inventory_status',
 'inventory_type',
 'is_fresh',
 'seller',
 'is_gift_card',
 'data_version',
 'brand',
 'current_seller',
 'other_sellers',
 'categories',
 'has_freeship_plus_benefit',
 'partner_rewards_amount',
 'tiki_rewards_amount',
 'total_rewards_amount',
 'product_virtual_type',
 'fulfillment_type']

In [14]:
####DUMP DATA TO FILE
def dump_to_file(data,filename:str):
    with open(f'Data/{filename}.csv', 'a') as f:
        writing_data= csv.writer(f, delimiter=',')
        writing_data.writerow(data)

#### TESTING FUNC

**DUMP DATA TO MYSQL**

In [15]:
#import thư viện
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

In [16]:
###create engine
#define
username = 'DT47_UIT'
password = '198198'
host = 'localhost'

DB_NAME = 'Tiki_Product'

# Create a connection object
databaseServerIP            = host
databaseUserName            = username
databaseUserPassword        = password
newDatabaseName             = DB_NAME
charSet                     = "utf8mb4"
cusrorType                  = pymysql.cursors.DictCursor

In [17]:
connectionInstance = pymysql.connect(host=databaseServerIP, user=databaseUserName, password=databaseUserPassword, charset=charSet, cursorclass=cusrorType)
connectionInstance

In [18]:
try:
    # Create a cursor object
    cursorInsatnce = connectionInstance.cursor()                                    
    # SQL Statement to create a database
    sqlStatement = "CREATE DATABASE IF NOT EXISTS "+ newDatabaseName  
    # Execute the create database SQL statment through the cursor instance
    cursorInsatnce.execute(sqlStatement)

except Exception as e:
    print("Exception occured:{}".format(e))
finally:
    connectionInstance.close()

**CREATE TABLE IF NOT EXISTS**

In [19]:
def create_table(database: str, table: str, datatype: str):
    try:
        username = 'DT47_UIT'
        password = '198198'
        host = 'localhost'
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        with engine.connect() as conn:
            conn.execute(text(f'DROP TABLE IF EXISTS {table};'))
            conn.execute(text(f'CREATE TABLE {table}({datatype})'))
            print('Table is created....')
    except SQLAlchemyError as e:
        print("Error while connecting to MySQL", e)

In [20]:
table = "seller"
datatype = '''id INT(255),
            name varchar(255),
            store_id INT(255),
            is_best_store BOOLEAN,
            PRIMARY KEY (id)'''
create_table("Tiki_Product", table=table, datatype=datatype)

Table is created....


**CREATE CATEGORIES TABLE**

In [21]:
table = "categories"
datatype = '''url varchar(255),
            name varchar(255),
            category_id INT(255),
            PRIMARY KEY (category_id)'''
create_table("Tiki_Product", table=table, datatype=datatype)

Table is created....


**CREATE PRODUCT TABLE**

In [22]:
table = "product"
datatype = '''id INT(255),
            name VARCHAR(255),
            short_url VARCHAR(255),
            short_description VARCHAR(255),
            price BIGINT(255),
            list_price BIGINT(255),
            original_price BIGINT(255),
            discount INT(255),
            discount_rate FLOAT(255, 2),
            rating_average FLOAT(255,2),
            inventory_status VARCHAR(255),
            inventory_type VARCHAR(255),
            is_fresh tinyint(1),
            seller VARCHAR(255) ,
            is_gift_card tinyint(1),
            data_version INT(255),
            brand INT(255),
            current_seller INT(225) default NULL,
            other_sellers VARCHAR(255),
            categories LONGTEXT,
            has_freeship_plus_benefit VARCHAR(255),
            partner_rewards_amount FLOAT(255,2),
            tiki_rewards_amount FLOAT(255,2),
            total_rewards_amount FLOAT(255,2),
            product_virtual_type VARCHAR(255),
            fulfillment_type VARCHAR(255),
            PRIMARY KEY (id)'''
create_table("Tiki_Product", table=table, datatype=datatype)

Table is created....


In [23]:
#test
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/Tiki_Product")
with engine.connect() as conn:
    A = conn.execute(text(f'SELECT DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = \'Tiki_Product\' AND table_name = \'{table}\' AND COLUMN_NAME = \'is_fresh\' ;'))
    print(A.fetchall())

[('tinyint',)]


In [24]:
#define read df 
def read_csv_to_dataframe(filename):
    df = pd.read_csv(f'Data/{filename}.csv', index_col=False, delimiter = ',')
    df = df.replace({np.NaN:None})
    return df
#test
read_csv_to_dataframe('product')

,id,name,short_url,short_description,price,list_price,original_price,discount,discount_rate,rating_average,...,is_fresh,seller,is_gift_card,data_version,brand,current_seller,other_sellers,inventory,breadcrumbs,amplitude


**Save to sql**

In [25]:
def import_categories(Dataframe:pd.DataFrame, database):
    try:                           #the parameters were declared at connect
        Dataframe.drop(columns = 'status')   
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        with engine.begin() as cnx:
            Dataframe.to_sql('categories', con = engine.connect(), if_exists= 'replace')                
    except Error as e:
            print("Error while import categories to MySQL", e)


In [26]:
def create_product_df(id):
    data, categories = get_data(id)
    f_data = extract_data_to_feild(data, f_product)
    df = extract_product_data(f_data)
    df = df.fillna(np.nan)  # Điền giá trị NaN cho các ô trống
    return df, categories

#test
df, cate = create_product_df(197553025)
df

,id,name,short_url,short_description,price,list_price,original_price,discount,discount_rate,rating_average,...,brand,current_seller,other_sellers,categories,has_freeship_plus_benefit,partner_rewards_amount,tiki_rewards_amount,total_rewards_amount,product_virtual_type,fulfillment_type
0,197553025,Điện thoại Nokia G11 Plus - Hàng Chính Hãng,https://tiki.vn/product-p197553025.html,"Thiết kế mỏng nhẹ, trẻ trung với màn hình 6.5 ...",3190000,3190000,0,0,0,4.5,...,19673,0,[],"[{'url': '/dien-thoai-may-tinh-bang/c1789', 'n...",False,0,4.903678,4.903678,NaN,tiki_delivery


In [27]:
product_ids = [114092342, 114766715, 120295859, 123345348]


In [28]:
import pandas as pd
import json
from sqlalchemy import create_engine

def import_data_mysql(Dataframe: pd.DataFrame, database: str):
    try:
        engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")
        
        # Xử lý các giá trị list và từ điển
        Dataframe = Dataframe.copy()
        Dataframe['categories'] = Dataframe['categories'].apply(lambda x: json.dumps(x) if isinstance(x, list) else None)
        Dataframe['other_sellers'] = Dataframe['other_sellers'].apply(lambda x: json.dumps(x) if isinstance(x, list) else None)
        
        Dataframe.to_sql('temporary_table', con=engine, if_exists='replace', index=False)
        with engine.begin() as cnx:
            insert_sql = text('INSERT IGNORE INTO product (SELECT * FROM temporary_table)')
            cnx.execute(insert_sql)
    except Exception as e:
        print("Error while import data to MySQL:", e)


In [29]:
for product_id in product_ids:
    df, cate = create_product_df(product_id)
    if not df.empty:
        import_data_mysql(df, "Tiki_Product")
    else:
        print(f"Failed to create dataframe or import product with id {product_id}")


In [30]:
p_id = [101615454,114092342,114766715,120295859,123345348]
df, categories = create_product_df(57809866)
print(categories)
for i in p_id:
    df1, categories = create_product_df(i)
    df = pd.concat([df, df1], ignore_index=True)
df.drop_duplicates(subset=['id'], inplace=True)
df = df.applymap(str)
import_data_mysql(df, 'Tiki_Product')


[{'url': '/dien-thoai-may-tinh-bang/c1789', 'name': 'Điện Thoại - Máy Tính Bảng', 'category_id': 1789}, {'url': '/dien-thoai-smartphone/c1795', 'name': 'Điện thoại Smartphone', 'category_id': 1795}, {'url': 'https://tiki.vn/dien-thoai-xiaomi-redmi-9a-2gb-32gb-hang-chinh-hang-p57809866.html', 'name': 'Điện thoại Xiaomi Redmi 9A (2GB/32GB) - Hàng chính hãng', 'category_id': 0}]


In [31]:
### Testing the Functional
def add_status_categories(categories,status):
    def change_status(x):
        x["status"] = status
        return x
    categories = list(map(change_status, categories))
    return pd.DataFrame.from_records(categories)


**TEST **

In [32]:
### Testing the Functional
def add_status_categories(categories,status):
    def change_status(x):
        x["status"] = status
        return x
    categories = list(map(change_status, categories))
    return pd.DataFrame.from_records(categories)


**THREAD ADD DATA TO PRODUCT DATA TABLE**


Create categories dict to store categories id : 100
multiple thread:
    + get data from categories and dump to database
    + get categories from data get and return to categories}

In [33]:
def add_categories_data(categories):
	global categories_df
	df1 = add_status_categories(categories,True)
	print('debug1')
	print(categories_df)
	categories_df = pd.concat([categories_df, df1], ignore_index=True)
	print('debug2')
	print(categories)

In [34]:
def thread_categories(lock):
    global categories_df
    global categories_list
    if categories_list != []:
        categories_filter = list(filter(lambda x: x['category_id'] not in categories_df['category_id'].unique(), categories_list))
        print(categories_filter, categories_list)
        add_categories_data(categories_filter)
        categories_list = []
    if len(categories_df.index) > 100:
        import_categories(categories_df, 'Tiki_Product')


In [35]:
def thread_add_product(lock):
    global categories_df
    global categories_list
    if len(categories_df.loc[(categories_df['status'] == True)]) > 0:
        lock.acquire()
        category = categories_df.loc[(categories_df['status'] == True)].iloc[1]
        if category is not None:
            Name = category['name']
            Id = category['category_id']
            max_page_number = get_max_page_number(Name, Id)
            p_ids = crawling_product_id(Name, Id, max_page_number)
            df, categories = create_product_df(p_ids[0])
            for i in p_ids[1:]:
                df1, categories = create_product_df(i)
                categories_list = categories_list + categories
                df = pd.concat([df, df1], ignore_index=True)  # Gộp DataFrame lại với nhau
            df.drop_duplicates(subset=['id'])
            df = df.applymap(str)
            categories_df.loc[categories_df['status'] == True].loc[1, 'status'] = False
            import_data_mysql(df, 'Tiki_Product')
            lock.release()

In [36]:
def main_task():
	global categories_df
	global categories_list
	print(categories_df)

	
	# creating a lock
	lock = threading.Lock()

	# creating threads
	t2 = threading.Thread(target=thread_add_product, args=(lock,))
	t1 = threading.Thread(target=thread_categories, args=(lock,))

	# start threads
	t1.start()
	t2.start()

	# wait until threads finish their job
	t1.join()
	t2.join()

if __name__ == "__main__":
	categories = get_product_catagories('https://tiki.vn/do-choi-me-be/c2549')
	categories_df = add_status_categories(categories,  True)
	categories_list = []
	while (len(categories_df.index) < 100):
		main_task()


Exception in thread Thread-5 (thread_add_product):
Traceback (most recent call last):
  File "C:\Users\DT_UIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexes\base.py", line 3652, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'status'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []


Exception in thread Thread-25 (thread_add_product):
Traceback (most recent call last):
  File "C:\Users\DT_UIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexes\base.py", line 3652, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'status'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []


Exception in thread Thread-49 (thread_add_product):
Traceback (most recent call last):
  File "C:\Users\DT_UIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexes\base.py", line 3652, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'status'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8

...

Exception in thread Thread-697 (thread_add_product):
Traceback (most recent call last):
  File "C:\Users\DT_UIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexes\base.py", line 3652, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'status'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []


Thread-723 (thread_add_product):
Traceback (most recent call last):
  File "C:\Users\DT_UIT\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pandas\core\indexes\base.py", line 3652, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\index.pyx", line 147, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 176, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 7080, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 7088, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'status'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.11_3.11.1264.0_x64__qbz5n2kfra8p0\Lib\threading.py

Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []


KeyboardInterrupt: 